In [25]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.circuit.library import QFTGate  
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import Sampler as AerSampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import numpy as np
from qiskit.circuit.library.basis_change import QFT
from neqr import neqr
from ladder import ladder_up
from enhanced_grover import filter_oracle
from qsubtract import subtract
from qadd import qft_adder
from PIL import Image
from qedgenod import qedgevh
import os

In [26]:
num_bits = 3
intensity_bits =  2
dim = 2 ** num_bits

In [27]:
img_file = 'square.png'
base_name = os.path.splitext(os.path.basename(img_file))[0]
output_dir = f"{base_name}_edges"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
def load_image(filepath, size):
    img = Image.open(filepath).convert('L')  # Grayscale
    img = img.resize(size, Image.Resampling.LANCZOS)
    img_array = np.array(img)
    return img_array

def normalize_image(image, bits=intensity_bits):
    max_val = 2**bits - 1
    return np.round((image / 255) * max_val).astype(int)
    
#image = load_image(img_file,size=(dim,dim))
#image = normalize_image(image, bits=intensity_bits)
image = np.array([
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0],
    [0,0,1,1,1,1,0,0],
    [0,0,1,1,1,1,0,0],
    [0,0,1,1,1,1,0,0],
    [0,0,1,1,1,1,0,0],
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0]

])
print(image)
np.savetxt("normalized_image.txt", image, fmt='%d')


[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 1 1 1 1 0 0]
 [0 0 1 1 1 1 0 0]
 [0 0 1 1 1 1 0 0]
 [0 0 1 1 1 1 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]


In [29]:
max_threshold = 2**(intensity_bits+1) - 1

In [30]:
for t in range(max_threshold):
    threshold = format(t, f'0{intensity_bits + 1}b')
    print(f"\nRunning threshold: {threshold}")
        
        # 3. Run quantum edge detection
    counts = dict(qedgevh(image, threshold, num_bits, intensity_bits))
        
        # 4. Initialize edge image
    dim = 2**num_bits
    edge_img = np.zeros((dim, dim), dtype=int)

        # 5. Parse measurement results
    for qstate, count in counts.items():
        qstate = qstate.replace(" ", "")

        if len(qstate) < 1 + 2 * num_bits:
            print("error")
            continue

        intensity_bin = qstate[:1]
        y_bin = qstate[1:1 + num_bits]
        x_bin = qstate[1 + num_bits:1 + 2*num_bits]

        intensity = int(intensity_bin, 2)
        y = int(y_bin, 2)
        x = int(x_bin, 2)

        edge_img[y, x] = intensity

        # 6. Display output
    plt.imshow(edge_img, cmap='gray_r', interpolation='nearest')
    plt.colorbar(label='Intensity')
    plt.title(f'Edge Filtered Image\nThreshold: {threshold}')
    plt.axis('off')
    out_path = os.path.join(output_dir, f"{base_name}_threshold_{threshold}_size{num_bits}.png")
    np.savetxt(os.path.join(output_dir, f"{base_name}_threshold_{threshold}.txt"), edge_img, fmt='%d')

    plt.savefig(out_path, bbox_inches='tight')
    plt.close()  # Close the figure to avoid memory bloat 


Running threshold: 000
3
[0, 0, 0]
2 0
max 2
1 0
0 0

Running threshold: 001
3
[1, 0, 0]
2 0
max 2
1 0
0 1

Running threshold: 010
3
[0, 1, 0]
2 0
max 2
1 1
0 0

Running threshold: 011
3
[1, 1, 0]
2 0
max 2
1 1
0 1

Running threshold: 100
3
[0, 0, 1]
2 1
1 0
0 0

Running threshold: 101
3
[1, 0, 1]
2 1
1 0
0 1

Running threshold: 110
3
[0, 1, 1]
2 1
1 1
0 0
